<a href="https://colab.research.google.com/github/mostafa-ja/LLM_from_scratch/blob/main/GPT_2_small_dataset_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model

In [5]:
import os
import math
import time
import inspect
import torch
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
from dataclasses import dataclass
from tqdm import tqdm
from torch.nn.utils import clip_grad_norm_

In [6]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer('bias', torch.tril(torch.ones(config.block_size, config.block_size))
                                    .view(1,1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        qkv = self.c_attn(x) # shape: (B, T, 3 * C)
        q, k, v = qkv.split(self.n_embd, dim=2) # Each of shape (B, T, C)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        #att = (q @ k.transpose(-2,-1)) * (1.0 / math.sqrt(k.size(-1)))
        #att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        #att = F.softmax(att, dim=-1)
        #y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)

        # flashattention
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        y = y.transpose(1,2).contiguous().view(B, T, C) # (B, nh, T, hs) → (B, T, nh, hs) → (B, T, nh * hs) = (B, T, C)
        y = self.c_proj(y)

        return y


class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu    = nn.GELU(approximate='tanh')
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd)
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer["wte"].weight = self.lm_head.weight

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, 'NANOGPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"

        # forward the token and position embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device).unsqueeze(0)  # shape (1, T)
        pos_emb = self.transformer["wpe"](pos)               # (1, T, n_embd)
        tok_emb = self.transformer["wte"](idx)               # (B, T, n_embd)
        x = tok_emb + pos_emb                                # broadcasting (B, T, n_embd)

        # forward the blocks of the transformer
        for block in self.transformer["h"]:
            x = block(x)

        # final layernorm and output projection
        x = self.transformer["ln_f"](x)
        logits = self.lm_head(x)                             # (B, T, vocab_size)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, device_type):

        # Select parameters that require gradients
        param_dict = {pn: p for pn, p in self.named_parameters() if p.requires_grad}

        # Separate parameters for weight decay
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]

        # Display parameter distribution
        print(f"decay param tensors: {len(decay_params)}, total: {sum(p.numel() for p in decay_params):,}")
        print(f"no-decay param tensors: {len(nodecay_params)}, total: {sum(p.numel() for p in nodecay_params):,}")

        # Use fused AdamW if supported and on CUDA
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        print(f"Using fused AdamW: {use_fused}")

        optimizer = torch.optim.AdamW(
            optim_groups,
            lr=learning_rate,
            betas=(0.9, 0.95),
            eps=1e-8,
            fused=use_fused
        )
        return optimizer


# Dataset

In [ ]:
#!pip install transformers
!pip install tiktoken

In [ ]:
import os
import tiktoken
#from transformers import AutoTokenizer#LLaMA 3 tokenizer

In [ ]:
def get_tokenizer(model_desc):
    """Returns tokenizer function and end-of-text token based on model."""
    if model_desc == "gpt-2":
        enc = tiktoken.get_encoding("gpt2")
        encode = lambda s: enc.encode_ordinary(s)
        eot_token = enc._special_tokens['<|endoftext|>']
    elif model_desc == "llama-3":
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B")
        encode = lambda s: tokenizer.encode(s, add_special_tokens=False, verbose=False, split_special_tokens=True)
        eot_token = tokenizer.encode('')[0]  # Adds EOT token by default
    else:
        raise ValueError(f"Unknown model descriptor: {model_desc}")
    return encode, eot_token

def process_text_file(filepath, encode, eot_token, train_split=0.9):
    """Reads text, tokenizes with EOT prepended to each section, and splits into train/val."""
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    # Splits the text into chunks wherever there are double newlines (\n\n), treating each as a separate "document" and Remove those \n\n separators.
    sections = text.split("\n\n")
    tokens = []

    for i, section in enumerate(sections):
        tokens.append(eot_token)  # Prepend EOT to mark document start
        padded_section = section + "\n\n" if i != len(sections) - 1 else section
        tokens.extend(encode(padded_section))

    split_idx = int(train_split * len(tokens))
    train_tokens = tokens[:split_idx]
    val_tokens = tokens[split_idx:]
    return train_tokens, val_tokens

In [ ]:
class DataLoaderLite:
    def __init__(self, tokens, batch_size, block_size):
        self.tokens = torch.tensor(tokens, dtype=torch.long)
        self.batch_size = batch_size
        self.block_size = block_size

        self.total_tokens = len(self.tokens)
        self.tokens_per_epoch = (self.total_tokens - 1) // (batch_size * block_size)
        assert self.tokens_per_epoch > 0, "Tokens per epoch must be positive"

        # Truncate to a clean multiple
        usable_tokens = self.tokens_per_epoch * batch_size * block_size + 1
        self.tokens = self.tokens[:usable_tokens]

        self.current_position = 0
        print(f"1 epoch in data loader = {self.tokens_per_epoch} batches")

    def next_batch(self):
        B, T = self.batch_size, self.block_size
        start = self.current_position
        end = start + B * T + 1

        if end > len(self.tokens):
            self.current_position = 0
            start, end = 0, B * T + 1

        buf = self.tokens[start:end]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += B * T
        return x, y

    def __next__(self):
        if self.current_position + self.batch_size * self.block_size + 1 > len(self.tokens):
            print('Next epoch ....')
        return self.next_batch()


# Training

In [7]:
# ==== Set Seed for Reproducibility ====
def set_seed(seed=1337):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [8]:
import math

def get_lr(step, max_lr = 6e-4, warmup_steps = 10, max_steps= 1000):
    """
    Compute learning rate using linear warmup followed by cosine decay.

    Args:
        step (int): Current training step.
        max_lr (float): Peak learning rate after warmup.
        min_lr (float): Final learning rate after decay (default: 10% of max_lr).
        warmup_steps (int): Number of steps for linear warmup.
        max_steps (int): Total number of training steps.

    Returns:
        float: Learning rate at the current step.
    """
    min_lr = max_lr * 0.1

    if step < warmup_steps:
        # Linear warmup
        return (max_lr * (step + 1) / warmup_steps)

    if step >= max_steps:
        # After training ends, return floor LR
        return min_lr

    # Cosine decay
    decay_ratio = (step - warmup_steps) / (max_steps - warmup_steps)
    decay_ratio = min(max(decay_ratio, 0.0), 1.0)  # Clamp between [0, 1]
    cosine_decay = 0.5 * (1 + math.cos(math.pi * decay_ratio))
    lr = min_lr + (max_lr - min_lr) * cosine_decay
    return lr

In [10]:
os.makedirs('dataset', exist_ok=True)
!wget -O dataset/tiny_shakespeare.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

model_desc = "gpt-2"
data_path = os.path.join('dataset', "tiny_shakespeare.txt")

encode_fn, eot = get_tokenizer(model_desc)
train_tokens, val_tokens = process_text_file(data_path, encode_fn, eot)



--2025-05-07 17:15:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘dataset/tiny_shakespeare.txt’

dataset/tiny_shakes 100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-05-07 17:15:57 (26.4 MB/s) - ‘dataset/tiny_shakespeare.txt’ saved [1115394/1115394]



In [11]:
@dataclass
class GPTConfig:
    block_size: int = 1024 # max sequence length
    vocab_size: int = 50304 # number of tokens, instead of 50257
    n_layer: int = 12 # number of transformer layers
    n_head: int = 12 # number of heads in multi-head attention
    n_embd: int = 768 # embedding dimension

In [ ]:
import time

# ==== Initialize ====
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
torch.set_float32_matmul_precision('high')
print(f"Current precision: {torch.get_float32_matmul_precision()}")
set_seed(1337)


total_batch_size = 524288  # 2**19 ~ 0.5M used for nice number
T = 1024  # Sequence length   (real is 1024)
B = 4  # Micro batch size (real is 16)
max_lr = 6e-4
max_steps = 50

assert total_batch_size % (B * T) == 0, "Total batch size must be divisible by micro batch size and sequence length"
grad_accum_steps = total_batch_size // (B * T)
print(f"Total batch size: {total_batch_size}")
print(f"Gradient accumulation steps: {grad_accum_steps}")

train_loader = DataLoaderLite(train_tokens, B, T)
val_loader = DataLoaderLite(val_tokens, B, T)


# ==== Model and Optimizer ====
model = GPT(GPTConfig()).to(device)
model = torch.compile(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
optimizer = model.configure_optimizers(
    weight_decay=0.1,
    learning_rate=max_lr,
    device_type=device
)



model.train()
for step in range(max_steps):
    start_time = time.time()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_batch in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(device), y.to(device)

        # Use autocast to automatically cast the model to the correct precision
        with torch.autocast(device, dtype=torch.bfloat16):
            logits, loss = model(x, y)

        # We need to scale our loss by the number of micro batches to get the correct gradient
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        # By keeping the loss in the loop, we can accumulate the gradients over the micro batches
        loss.backward()

    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    lr = get_lr(step, max_lr=max_lr, warmup_steps=10, max_steps=max_steps) #int(max_steps * 0.01)
    for group in optimizer.param_groups:
        group["lr"] = lr

    # Outside the microbatch loop so all accumulated update in one go
    optimizer.step()
    torch.cuda.synchronize()
    elapsed = time.time() - start_time
    tokens_per_sec = (B * T * grad_accum_steps) / elapsed
    print(f"Step {step + 1:5d} | Loss: {loss_accum:.4f} | LR: {lr:.2e} | GradNorm: {grad_norm:.2f} | Tokens/sec: {tokens_per_sec:.2f}")


Using device: cuda
Current precision: high
Total batch size: 524288
Gradient accumulation steps: 128
1 epoch in data loader = 74 batches
1 epoch in data loader = 8 batches
Total parameters: 124,475,904
decay param tensors: 50, total: 124,354,560
no-decay param tensors: 98, total: 121,344
Using fused AdamW: True


W0507 17:17:40.584000 12417 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Step     1 | Loss: 10.9876 | LR: 6.00e-05 | GradNorm: 2.36 | Tokens/sec: 2463.78
Step     2 | Loss: 10.3814 | LR: 1.20e-04 | GradNorm: 2.52 | Tokens/sec: 2588.20
Step     3 | Loss: 9.6795 | LR: 1.80e-04 | GradNorm: 2.53 | Tokens/sec: 2565.79
